In [2]:
library(plyr)
library(magrittr)
library(distr)
library(distrEx)
library(gbm)
library(Matching)
library(caret)
library(tidyverse)

In [3]:
DGP = list()
n = 5

X1 = Norm(-1)
X2 = Norm(1)
DGP$X = list(X1, X2)

DGP$f_W_x = function(x, w) {
    logit_p = x[1] + x[2]
    p = exp(logit_p) / (1 + exp(logit_p))
    Binom(prob=p)
}

DGP$f_Y_xw = function(x, w) {
    if(w) {
        Weibull(scale=abs(x[1] + x[2]) + 0.7, 
                shape=1.2)
    } else {
        Weibull(scale= abs(x[1]) + abs(x[2]), 
                shape=1.5)
    }
}

DGP$f_C_xw = function(x, w) {
    Weibull(scale=4, 
            shape=1.4)
}

In [4]:
#' @import dplyr
#' @import purrr
#' @import tidyr
#' @import magrittr
#' @import stringr
#' @import caret
#' @import Matching
#' @import distr
#' @import distrEx

## Each patient will now get their own control Y0 and treatment Y1 distributions from 
# package distr, with parameters f(X) for more or less arbitrary f()
# Then we'll calculate tau as E[Y1] - E[Y0] using the E() function
# from distrEx. 
# yi = w_i*r(Y1) + (1-w_i)*r(Y0)
# 

data_list_to_df = function(data_list) {
    data_list$covariates %<>% data.frame %>% set_names(paste("covariate", 1:ncol(.), sep="_"))
    data_list %$% cbind(subject, event, time, treatment, covariates) %>% data.frame
}

data_df_to_list = function(data_df) {
    data_list = list()
    data_list$subject = data_df %>% pull(subject)
    data_list$outcome = data_df %>% pull(outcome) # an indicator: 1 for death, 0 for censoring
    data_list$time = data_df %>% pull(time) # the time of either death or censoring
    data_list$treatment = data_df %>% pull(treatment)
    data_list$covariates = as.matrix(data_df %>% select(starts_with()))
    return(data_list)
}

#' Generate simulated observational data
#'
#' Generates N tuples from the joint distribution of P(X,W,Y).
#' Returns a list with two elements: a dataframe called data and a dataframe 
#' called aux_data containing true expecations conditional on X.
#' @param X a list of distribution objects that generate a covariate vector
#' @param f_W_x a function of x that returns the conditional distribution W|X=x
#' @param f_Y_xw a function of x, w that returns the conditional distribution Y|X=x,W=w
#' @param f_C_xw a function of x, w that returns the conditional distribution C|X=x,W=w
#' @keywords
#' @export
#' @examples
dgp = function() {
    list(X=X, f_W_x=f_W_x, f_Y_xw=f_Y_xw, f_C_xw=f_C_xw)
}
# These functions e.g. f_W_X will likely be generated on-the-fly:
# make_f_W_X_dist = function(f1, f2, ...) {
#       function(x) SomeDist(param1 = f1(x), param2 = f2(x)... )
# }

sample1 = function(dist) {
    r(dist)(1)
}

#' Generate simulated observational data
#'
#' Generates N tuples from the joint distribution of P(X,W,Y).
#' Returns a list with two elements: a dataframe called data and a dataframe 
#' called aux_data containing true expecations conditional on X.
#' @param DGP a data generating process, which is the list output of the dgp() function
#' @param n the number of desired samples
#' @keywords
#' @export
#' @examples
create_data = function(DGP, n=1) {
    x = n %>% rerun(DGP$X %>% map_dbl(sample1)) 

    W = x %>% map(DGP$f_W_x) 
    w = W %>% map_dbl(sample1)
    pw = W %>% map_dbl(E) # propensity scores

    Y1 = x %>% map(~DGP$f_Y_xw(.,1))
    Y0 = x %>% map(~DGP$f_Y_xw(.,0)) 
    mu1 = Y1 %>% map_dbl(E)
    mu0 = Y0 %>% map_dbl(E)
    tau = mu1 - mu0
    y = list(w,Y1,Y0) %>% 
        pmap_dbl(function(w, Y1, Y0) ifelse(w, sample1(Y1), sample1(Y0)))

    C1 = x %>% map(~DGP$f_C_xw(.,0))
    C0 = x %>% map(~DGP$f_C_xw(.,1))
    ce = list(w,C1,C0) %>% 
        pmap_dbl(function(w, C1, C0) ifelse(w, sample1(C1), sample1(C0)))

    t = pmin(y,ce)
    d = y < ce

    pc = list(w, t, C1, C0) %>% # censoring probability at t
         pmap_dbl(function(w,t,C1,C0) ifelse(w, 1-p(C1)(t), 1-p(C0)(t)))

    data = list()
    data$subject = 1:length(w)
    data$time = t
    data$event = d
    data$treatment = as.logical(w)
    data$covariates = x %>% reduce(rbind)
    rownames(data$covariates) = NULL

    aux_data = data.frame(
        subject=data$subject, 
        treated_mean=mu1, 
        control_mean=mu0, 
        effect=tau,
        iptw=1/(1-w + 2*w*pw - pw),
        ipcw=1/pc)

    return(list(data=(data %>% data_list_to_df), aux_data=aux_data))
}

f1 = function(x) rep(0, nrow(x))

f2 = function(x) 6 * (x[, 1] > 1) - 1 - 
    (6*pnorm(-1) -1) # take out the expectation

f3 = function(x) 5 * x[, 1]

f4 = function(x) {
    1 * x[,2] * x[,4] * x[,6] + 
    2 * x[,2] * x[,4] * (1-x[,6]) +
    3 * x[,2] * (1-x[,4]) * x[,6] + 
    4 * x[,2] * (1-x[,4]) * (1-x[,6]) +
    5 * (1-x[,2]) * x[,4] * x[,6] + 
    6 * (1-x[,2]) * x[,4] * (1-x[,6]) +
    7 * (1-x[,2]) * (1-x[,4]) * x[,6] + 
    8 * (1-x[,2]) * (1-x[,4]) * (1-x[,6]) - 
    5 -
    (-0.5) # take out the expectation
}

f5 = function(x) x[, 1] + x[, 3] + x[, 5] + x[, 7] + x[, 8] + x[, 9] -
    (0.5) # take out the expectation

f6 = function(x) {
    4 * (x[,1]>1) * (x[,3]>0) + 4 * (x[,5]>1) * (x[,7]>0) + 2 * x[,8] * x[,9] - 1 -
    (4*pnorm(-1)-1) # take out the expectation
}

f7 = function(x) {
  ( x[, 1]^2 + 
    x[, 2] + 
    x[, 3]^2 + 
    x[, 4] + 
    x[, 5]^2 + 
    x[, 6] + 
    x[, 7]^2 +
    x[, 8] + 
    x[, 9]^2 ) / 
    sqrt(2) - 5 -
    (7/sqrt(2) - 5) # take out the expectation
}

f8 = function(x) (f4(x) + f5(x)) / sqrt(2) 
    # ((-0.5 - )sqrt(2)) # take out the expectation

f9 = function(x) (x[,1])^2 - 1

#' Simulations 
#'
#' @export
schuler_DGPs = function() {
    X = list(Norm(-1), Norm(1))

    f_W_x = function(x, w) {
        logit_p = x[1] + x[2]
        p = exp(logit_p) / (1 + exp(logit_p))
        Binom(prob=p)
    }

    f_Y_xw = function(x, w) {
        if(w) {
            Weibull(scale=abs(x[1] + x[2]) + 0.7, 
                    shape=1.2)
        } else {
            Weibull(scale= abs(x[1]) + abs(x[2]), 
                    shape=1.5)
        }
    }

    f_C_xw = function(x, w) {
        Weibull(scale=4, 
                shape=1.4)
    }

    list("biased" = dgp(X, function(x,w) 0.5, f_Y_xw, f_C_xw), 
         "unbiased" = dgp(X, f_W_x, f_Y_xw, f_C_xw))
}

In [5]:
#' @import dplyr
#' @import purrr
#' @import tidyr
#' @import magrittr
#' @import caret
#' @import Matching

# setup_fitting_1_model = function(method, tune_grid=NULL, train_index) {
# 	function(data) train(
# 		  x = data %>% dplyr::select(treatment, starts_with("covariate")) %>% as.matrix,
# 		  y = data$outcome,
# 		  method = method,
# 		  trControl = trainControl(method='cv', 
#                                  number=length(train_index),
# 		  						 index=train_index,
#                                  returnResamp="all",
#                                  savePredictions="all"),
# 		  tuneGrid = tune_grid)
# }

# test_estimate_hte_1_model = function(data, method, tune_grid, train_index) {
# 	fitting_function = setup_fitting_1_model(method, tune_grid, train_index)
# 	cf_data = data %>% # must be in this order for the train index to work
#     	mutate(treatment = !treatment)
# 	full_data = bind_rows(data, cf_data)
# 	models = full_data %>% fitting_function
# 	models$pred %>% # this carries with it columns with all the values of the hyperparameters
# 		mutate(method = method) %>%
# 		# unite(model, -pred, -obs, -rowIndex, -Resample, sep="^") %>%
# 		unite_("model", c("method", names(tune_grid)), sep="~") %>%
# 		# unite(model, !!!syms(c("method", names(tune_grid))), sep="~") %>%
# 	    mutate(cf = ifelse(rowIndex > nrow(data), "counterfactual", "factual")) %>%
# 	    mutate(subject = ifelse(cf=="factual", rowIndex, rowIndex-nrow(data))) %>%
# 	    select(-rowIndex, -obs) %>%
# 	    spread(cf, pred) %>% 
# 	    arrange(Resample, subject) %>%
# 	    filter(!is.na(factual)) %>%
# 	    inner_join(data %>% select(subject, treatment, outcome), by="subject") %>%
# 	    mutate(treated=ifelse(treatment, factual, counterfactual),
# 	           control=ifelse(treatment, counterfactual, factual),
# 	           effect=treated-control)
# }


gbm_ph_fit_predict = function(x_train, y_train, x_val, rowIndex, interaction.depth, n.minobsinnode, shrinkage, n.trees) {
    model = gbm.fit(x_train, y_train, distribution="coxph", verbose=F,
                    n.trees=max(n.trees), interaction.depth=interaction.depth, 
                    shrinkage=shrinkage, n.minobsinnode=n.minobsinnode)
    predict(model, x_val, n.trees=n.trees) %>% 
        data.frame %>%
        mutate(rowIndex=rowIndex, interaction.depth=interaction.depth, 
               shrinkage=shrinkage, n.minobsinnode=n.minobsinnode) %>%
        gather(n.trees, pred, -rowIndex, -interaction.depth, -shrinkage, -n.minobsinnode) %>%
        mutate(n.trees = str_replace(n.trees,"X","") %>% as.numeric)
}

# the thing that is estimated by coxph models is the log relative (to the basline) risk 
fit_model = function(data, train_index, method, tune_grid) {
    x_train = data[train_index,] %>% dplyr::select(starts_with("covariate")) %>% as.matrix
    y_train = data[train_index,] %$% Surv(time, event)
    x_val = data[-train_index,] %>% dplyr::select(starts_with("covariate")) %>% as.matrix
    rowIndex = data[-train_index,] %>% pull(subject)
    
    if(method=="gbm") {
        grouped_tune_grid = tune_grid %>% 
            group_by(interaction.depth, n.minobsinnode, shrinkage) %>%
            summarize(n.trees=list(n.trees))
        grouped_tune_grid %>%
            pmap(function(interaction.depth, n.minobsinnode, shrinkage, n.trees) {
                gbm_ph_fit_predict(x_train, y_train, x_val, rowIndex,
                                   interaction.depth, n.minobsinnode, 
                                   shrinkage, n.trees)}) %>%
            bind_rows()
    }
}

prep_fold_data = function(training_data, test_data) {
	data = bind_rows(training_data, test_data) %>%
		mutate(rowIndex=row_number())
	index = list(c(data %>% 
					filter(sample_type=="training") %>% 
					pull(rowIndex)))
	return(list(data=data, index=index))
}

test_estimate_hte = function(data, method, tune_grid, fold, fold_name) {
	training_data = data[fold,] %>% mutate(sample_type="training")
	test_data = data[-fold,] %>% mutate(sample_type="test")

	fold_data = training_data %>% 
		split(.$treatment) %>%
		map(~prep_fold_data(., test_data)) #now have a list (treat => (data, fold))
	predictions = fold_data %>%
		map(~fit_model(.$data, .$index, method, tune_grid)$pred) # returns the big matrix with all test set predictions for each treatment
	test_estimates = fold_data %>%
	    map(~select(.$data, subject, treatment, time, event, rowIndex)) %>%
	    list(predictions) %>%
	    pmap(function(data, predictions) inner_join(data, predictions, by="rowIndex")) %>%
	    imap(function(df,name) rename_(df, .dots=setNames("pred", str_c("est_rel_risk", name, sep="_")))) %>% # df_TRUE$pred and df_FALSE$pred become (est_rel_risk_TRUE, ..._FALSE) in the same df
	    reduce(inner_join, by=c("subject", "treatment", "time", "event", names(tune_grid))) %>% # will join on all columns... if didn't want to join on params would also have to join across methods!
	    mutate(method=method) %>%
	    unite_("model", c("method", names(tune_grid)), sep="~") %>% 
	    mutate(fold=fold_name) %>%
	    mutate(est_effect=est_rel_risk_TRUE-est_rel_risk_FALSE, 
	    	   est_rel_risk=treatment*(est_rel_risk_TRUE) + (1-treatment)*est_rel_risk_FALSE) %>% # selects the estimated time, event from the appropriate model
	    select(subject, model, treatment, time, event, est_effect, est_rel_risk, fold) 
}

cross_estimate_hte = function(data, method, tune_grid, train_index) {
	train_index %>%
	imap(function(fold, fold_name) test_estimate_hte(data, method, tune_grid, fold, fold_name)) %>%
	bind_rows()
}

In [6]:
#' @import dplyr
#' @import purrr
#' @import tidyr
#' @import magrittr
#' @import caret
#' @import Matching

# group data into all test folds
# for each test fold, do the internal matching
# record the matched patient
find_matches = function(data) {
	treated_match = Match(Tr=data$treatment, 
	   					  X=data.matrix(data%>%select(starts_with("covariate"))),
	    				  replace=T, estimand="ATT") 
	control_match = Match(Tr=!data$treatment, # (separate so one treated doesn't get two controls ambiguously)
	    				  X=data.matrix(data%>%select(starts_with("covariate"))),
	    				  replace=T, estimand="ATT")
	data.frame(subject = data$subject[c(treated_match$index.treated, control_match$index.treated)], # all subjects
			   match = data$subject[c(treated_match$index.control, control_match$index.control)]) # their matches
}

create_cv_index = function(data, n_folds=5) {
		createFolds(data$treatment, k=n_folds) %>% # hold-out indices
    	map(~(data$subject)[-.]) #  complement of each index, in terms of the subject IDs
}


# see:
# Fewell, Z., Hernán, M. A., Wolfe, F., Tilling, K., Stata, H. C., 2004. (n.d.). Controlling for time-dependent confounding using marginal structural models. Pdfs.Semanticscholar.org
# Rodriguez, G. (2014). Survival Models. In Lecture Notes on Generalized Linear Models (pp. 1–34).
ipcw = function(data, n_time_intervals=10) {
	interval_data = data %>% 
	    mutate(event = !event) %>% # "event" is now censoring, not death
	    mutate(event_interval = ntile(time, n_time_intervals))
	pseudo_data = list(subject = interval_data %$% unique(subject), 
                   interval = interval_data %$% unique(event_interval)) %>%
    cross_df %>%
    inner_join(interval_data, by="subject") %>%
    filter(interval <= event_interval) %>% # keep only pseudo-observations before or at event time
    mutate(event = ifelse(interval==event_interval, event, FALSE)) # before the real time of censoring, set censoring=F
    pCt_XW = glm.fit( # no global intercept
	    x = pseudo_data %>% 
	            select(subject, interval, treatment, starts_with("covariate")) %>% 
	            mutate(interval_copy = interval) %>%
	            mutate(trash=1) %>%
	            spread(interval, trash, fill=0) %>% # give each interval its own intercept
	            select(-interval_copy, -subject) %>%
	            data.matrix(),
	    y = pseudo_data %>% pull(event),
	    family = binomial(link="cloglog")) %$% 
		fitted.values
    data.frame(pCt_XW = pCt_XW,
          	   subject = pseudo_data$subject) %>%
	    group_by(subject) %>%
	    summarize(cens_prob_at_event_time = prod(1-pCt_XW)) %>%
	    ungroup() %>%
	    mutate(est_ipcw = 1/cens_prob_at_event_time) %>%
	    inner_join(data, by="subject") %>%
	    select(subject, est_ipcw)
}

iptw = function(data) {
	pW_X = glm.fit(
		x = data %>% select(starts_with("covariate")) %>% as.matrix,
		y = data %>% pull("treatment"),
 		family = binomial(link="logit")) %$% 
	fitted.values
	data.frame(pW_X = pW_X, 
			   treatment=data$treatment,
			   subject=data$subject) %>%
		mutate(est_iptw = 1/(1-treatment + 2*treatment*pW_X - pW_X)) %>%
		select(subject, est_iptw)
}

#' Prepares simulated data for experiments
#'
#' @param DGP a list created by a call to dgp()
#' @param n_train number of samples to train and validate on
#' @param n_test number of samples to test on
#' @param n_folds the number of folds to use for treatment effect cross-validation
#' @keywords
#' @export
#' @examples
setup_data = function(DGP, n_train, n_test, n_folds) {
	simulation = create_data(DGP, n_train+n_test) 
	data = simulation$data %>% 
	    mutate(set = ifelse(subject > n_train, "test", "training"))
	aux_data = simulation$aux_data %>%
		mutate(set = ifelse(subject > n_train, "test", "training"))

	cv_index = data %>% 
	    filter(set=="training") %>%
	    create_cv_index(n_folds=n_folds)
	test_index = list("training" = (data %>% filter(set=="training") %>% pull(subject)))

	cv_held_out = cv_index %>% 
    	map(~test_index$training[!(test_index$training %in% .)])
	held_out = c(cv_held_out, list("training" = (data %>% filter(set=="test") %>% pull(subject))))
	matches = held_out %>% # need to find matches for each individual within each training fold... 
	    map(~ data.frame(subject=.) %>%
	    	inner_join(data, by="subject") %>% 
	    	find_matches()) %>%
	    bind_rows(.id="fold")

	aux_data = aux_data %>%
		inner_join(matches, by="subject") %>%
		inner_join(data %>% select(subject, treatment), by='subject') %>%
		inner_join(iptw(data), by="subject") %>% 	# I'm cheating a little bit here because I use test data to fit these
		inner_join(ipcw(data), by="subject") %>% 	# but the models should be underfit anyways so that data shouldn't add much...
		select(-treatment) # can fix this later but won't really change it
	return(list(data=data, aux_data=aux_data, cv_index=cv_index, test_index=test_index))
}

#' Estimates conditonal mean regression models on the data via caret. 
#' Does cross-estimation on the training data and runs each model trained on the full training set on the test set.
#' Returns all out-of-sample estimates from each model
#'
#' @param data  data 
#' @param models  models 
#' @param cv_index cv_index
#' @param test_index  test_index 
#' @keywords
#' @export
#' @examples
get_estimates = function(data, models, cv_index, test_index) {
	training_data = data %>% 
	    filter(set=="training") 
	cv_estimates = models %>% # cross validate on CV data (ignore the test set)
		map(~cross_estimate_hte(training_data, .$method, .$tune_grid, cv_index)) %>%
	    bind_rows() 
	test_estimates = models %>% # now train on all CV data and test on the test set
		map(~cross_estimate_hte(data, .$method, .$tune_grid, test_index)) %>% 
		bind_rows() 
	return(list(cv_estimates=cv_estimates, test_estimates=test_estimates))
}

compute_cv_metrics = function(estimates) {
	estimates  %>%
	    # dplyr::group_by(!!!syms(c(param_names, "fold"))) %>% # I do this for each fold
	    dplyr::group_by(model, fold) %>% # I do this for each fold
	    # mutate(est_effect_test_match = est_effect_covariate_matching(treatment, outcome, subject, match),
	    	   # est_effect_test_trans = est_effect_transformed_outcome(treatment, outcome, ip_weights)) %>%
	    dplyr::summarize(#### framework: ###
	    				 match_mse = est_effect_covariate_matching(treatment, outcome, subject, match) %>% loss_squared_error(est_effect),
	    				 trans_mse = est_effect_transformed_outcome(treatment, outcome, true_ip_weights) %>% loss_squared_error(est_effect), 
						 trans_mse_est_prop = est_effect_transformed_outcome(treatment, outcome, est_ip_weights) %>% loss_squared_error(est_effect), 
	    				 match_decision = est_effect_covariate_matching(treatment, outcome, subject, match) %>% loss_decision(est_effect),
	    				 trans_decision = est_effect_transformed_outcome(treatment, outcome, true_ip_weights) %>% loss_decision(est_effect), # aka gain!
	    				 trans_decision_est_prop = est_effect_transformed_outcome(treatment, outcome, est_ip_weights) %>% loss_decision(est_effect), # aka gain!
	    				 #### value: ####
						 value = -value(est_effect, treatment, outcome, weights=true_ip_weights),
						 gain = -gain(est_effect, treatment, outcome),
						 value_est_prop = -value(est_effect, treatment, outcome, weights=est_ip_weights),
						 # #### broken: ####
	    				 prediction_error = loss_squared_error(est_outcome, outcome),
	                     est_te_strata = est_effect_transformed_outcome(treatment, outcome, est_effect) %>% loss_squared_error(est_effect),
	                     #### ranking: ####
	                     c_benefit = -c_benefit(est_effect, treatment, outcome),
	                     qini = -qini(est_effect, treatment, outcome, true_ip_weights),
	                     qini_est_prop = -qini(est_effect, treatment, outcome, est_ip_weights),
	                     value_auc = -value_auc(est_effect, treatment, outcome, true_ip_weights),
	                     value_auc_est_prop = -value_auc(est_effect, treatment, outcome, est_ip_weights),
	                     ### random: ####
	                     random = random_metric()
	                     ) %>%
	   	dplyr::ungroup() %>% dplyr::select(-fold) %>% dplyr::group_by(model) %>% # Then average over the folds
	    dplyr::summarize_all(mean, na.rm=T)
}

compute_test_metrics = function(estimates) {
	estimates  %>%
	    dplyr::group_by(model) %>% # there should only be one fold
	    dplyr::summarize(true_hte_error = loss_squared_error(est_effect, true_effect),
	                     true_value = -true_value(est_effect, true_effect, true_mean)) 
}

#' Estimates estimated cv errors and true test errors (the latter via true values in aux_data). 
#'
#' @param cv_estimates  cv_estimates 
#' @param test_estimates test_estimates 
#' @param aux_data aux_data
#' @keywords
#' @export
#' @examples
get_errors = function(cv_estimates, test_estimates, aux_data) {
	cv_error = cv_estimates %>% 
		inner_join(aux_data, by=c("subject", "fold")) %>%
    	compute_cv_metrics() %>%
        gather(selection_method, error, -model)
	min_cv_error = cv_error %>%
	    group_by(selection_method) %>%
	    filter(error == min(error, na.rm=T)) %>%
	    sample_n(1) %>% # if there are ties for the lowest error, break at random
	    select(-error) %>% ungroup()
	test_error = test_estimates %>% 
		inner_join(aux_data, by=c("subject", "fold")) %>%
	    compute_test_metrics()
	oracle_error = test_error %>%
		gather(selection_method, error, -model) %>%
		group_by(selection_method) %>%
	    filter(error == min(error, na.rm=T)) %>%
	    sample_n(1) %>%
		select(-error) %>% ungroup() %>%
		mutate(selection_method = str_c("oracle_selector", selection_method, sep="_"))
	true_selection_error = min_cv_error %>%
		bind_rows(oracle_error) %>%
	    inner_join(test_error, by="model") %>%
	    bind_rows(data.frame(model="truth", selection_method="oracle", true_hte_error=0,  # this is the true model
	    					 true_value=-(aux_data %>% filter(set=="test") %$% true_value(true_effect, true_effect, true_mean)))) # this needs to be evaluated just over the test set!!!
	    # mutate(optimal_deficiency = -true_hte_value(true_effect, true_effect, true_mean)) # %>%
	    # mutate(scenario=scenario, n_folds=n_folds, training_percent=training_percent, rep=rep)
	return(list(cv_error=cv_error, test_error=test_error, true_selection_error=true_selection_error))
}


In [7]:
datas = setup_data(DGP, 500, 500, 3)

Warning message:
: package ‘bindrcpp’ was built under R version 3.2.5

In [18]:
models = list(
    gbm_spec = list(method = "gbm",
                tune_grid = expand.grid(n.trees = seq(1,501,20), 
                                        interaction.depth=3, 
                                        shrinkage = 0.2, 
                                        n.minobsinnode=3)))
herp = datas %$% get_estimates(data, models, cv_index, test_index)

ERROR: Error in xj[i]: invalid subscript type 'list'


In [15]:
method = "gbm"
tune_grid = expand.grid(n.trees = seq(1,501,20), 
                        interaction.depth=3, 
                        shrinkage = 0.2, 
                        n.minobsinnode=5)
train_index = 1:500

In [17]:
herp = fit_model(datas$data, datas$cv_index[[1]], method, tune_grid)

In [10]:
tail(derp)

,rowIndex,interaction.depth,shrinkage,n.minobsinnode,n.trees,pred
17337,995,3,0.2,5,501,3.187675
17338,996,3,0.2,5,501,-0.3993727
17339,997,3,0.2,5,501,-1.060509
17340,998,3,0.2,5,501,4.018024
17341,999,3,0.2,5,501,0.4351027
17342,1000,3,0.2,5,501,1.01304
